# 1. Definitions

In [14]:
basepath = r'E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-10-13'

In [2]:
%matplotlib inline
from pathlib import Path
import yaml
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 1.5})

In [3]:
import phconvert as phc
import polimi_tcspc
print(phc.__version__)

0.7.3+21.g9ade74b


In [4]:
def save_polimi_tcspc_to_photon_hdf5(filename, sample=None, setup=None, description=None, 
                                     validate=False):
    hardware = ('POLIMI-TCSPC module 16-CH [Pos1: 0403FE6A (ch 1-8), Pos3: 0403F16A (ch 9-16)]; '
                'Manta 0403DAE6.')
    if description is None:
        description = hardware

    filename = str(Path(filename))
    data, header_info = polimi_tcspc.loadfile(filename)
    timestamps_m = data['timestamps']
    nanotimes = data['nanotimes']
    channels = data['channels']
    timestamps_unit = 1/header_info['timestamps_clock_Hz']
    acquisition_duration = (np.max([t.max() for t in timestamps_m]) - 
                            np.min([t.min() for t in timestamps_m])) * timestamps_unit
    
    d = {}
    tcspc_num_bins = 2**14
    for ch in range(16):
        ch_mask = channels == ch
        tcspc_unit = header_info['tcspc_binwidths_ps'][ch]*1e-12

        photon_data = dict(
            timestamps = timestamps_m[ch],
            timestamps_specs = dict(timestamps_unit=timestamps_unit),
            detectors = channels[ch_mask],

            nanotimes = nanotimes[ch_mask],

            nanotimes_specs = dict(
                tcspc_unit = tcspc_unit,
                tcspc_range = tcspc_unit*tcspc_num_bins,
                tcspc_num_bins = tcspc_num_bins))

        d['photon_data%d' % ch] = photon_data

    d['description'] = description    
    d['acquisition_duration'] = acquisition_duration
    d['_filename'] = filename
    d['identity'] = {}

    if sample is not None:
        d['sample'] = sample
    
    d['setup'] = {'num_pixels': header_info['num_channels'], 
                  'num_spots': header_info['num_channels'], 
                  'num_spectral_ch': 1, 'num_polarization_ch': 1, 'num_split_ch': 1,
                  'modulated_excitation': False, 'lifetime': True,
                  'excitation_wavelengths': [532e-9], 'excitation_cw': [False],
                  'detection_wavelengths': [580e-9],
                  'excitation_alternated': (False,),
                  'user': {'name': hardware}}
    if setup is not None:
        d['setup'].update(setup)
    
    phc.hdf5.save_photon_hdf5(d, overwrite=True, validate=validate, skip_measurement_specs=False)

# 2. Check FIFO-full flags

In [43]:
basepath = r'E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-10-16'

In [44]:
for file in Path(basepath).glob('**/*.dat'):
    if not polimi_tcspc.is_polimi_tcspc(str(file)): 
        continue

    print('\n - %s' % Path(*file.parts[3:]), flush=True)
    fifo_full = polimi_tcspc.get_fifo_full_array(str(file))
    
    if fifo_full.any():
        print('     * FIFO Full!', flush=True)


 - data\POLIMI-TCSPC-16ch\2015-10-16\Cy3B-10X-ND2.4_4lambda_waveplate_3.dat

 - data\POLIMI-TCSPC-16ch\2015-10-16\Cy3B-10X-ND2.4_NO_4lambda_waveplate_4.dat

 - data\POLIMI-TCSPC-16ch\2015-10-16\ErythrosineB-H2O-filtered-saturated-KI-quenched-ND3_2.dat

 - data\POLIMI-TCSPC-16ch\2015-10-16\ErythrosineB-H2O-filtered-saturated-ND2.4_1.dat


In [34]:
basepath = r'E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-10-13'

In [15]:
for file in Path(basepath).glob('**/*.dat'):
    if not polimi_tcspc.is_polimi_tcspc(str(file)): 
        continue

    print('\n - %s' % Path(*file.parts[3:]), flush=True)
    fifo_full = polimi_tcspc.get_fifo_full_array(str(file))
    
    if fifo_full.any():
        print('     * FIFO Full!', flush=True)


 - data\POLIMI-TCSPC-16ch\2015-10-13\ErythrosinB_high_conc_1.dat

 - data\POLIMI-TCSPC-16ch\2015-10-13\ErythrosinB_sample2_3.dat
     * FIFO Full!

 - data\POLIMI-TCSPC-16ch\2015-10-13\IRF-glass-air-BP-ND3_5.dat

 - data\POLIMI-TCSPC-16ch\2015-10-13\IRF-glass-air-LP_BP_4.dat

 - data\POLIMI-TCSPC-16ch\2015-10-13\IRF-glass-air-ND1_BP_2.dat


# 3. Convert a single file

In [34]:
basepath = r'E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-10-16'

In [35]:
datafile = Path(basepath, r'ErythrosineB-H2O-filtered-saturated-ND2.4_1.dat')

In [36]:
assert datafile.exists()

In [37]:
sample = {'sample_name': 'ErythrosineB in H2O saturated with KI for lifetime quenching.',
          'num_dyes': 1,
          'dye_names': 'ErythrosineB',
          'buffer_name': 'Filtered water saturated with KI'}

setup = {'excitation_input_powers': [400e-6]}

description = 'High-concentraction of ErythrosineB in KI saturated solution for testing short lifetimes.'

In [38]:
save_polimi_tcspc_to_photon_hdf5(datafile, sample=sample, setup=setup, description=description, 
                                 validate=True)

Saving: E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-10-16\ErythrosineB-H2O-filtered-saturated-ND2.4_1.hdf5
Photon-HDF5 WARNING: Missing field "author" in "/identity".
Photon-HDF5 WARNING: Missing field "author_affiliation" in "/identity".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data0".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data1".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data2".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data3".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data4".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data5".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data6".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data7".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data8".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data9".
Photon-HDF5

In [39]:
datafile = Path(basepath, r'ErythrosineB-H2O-filtered-saturated-KI-quenched-ND3_2.dat')

In [40]:
assert datafile.exists()

In [41]:
sample = {'sample_name': 'ErythrosineB in H2O saturated with KI for lifetime quenching.',
          'num_dyes': 1,
          'dye_names': 'ErythrosineB',
          'buffer_name': 'Filtered water saturated with KI'}

setup = {'excitation_input_powers': [100e-6]}

description = 'High-concentraction of ErythrosineB in KI saturated solution for testing short lifetimes.'

In [42]:
save_polimi_tcspc_to_photon_hdf5(datafile, sample=sample, setup=setup, description=description, 
                                 validate=True)

Saving: E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-10-16\ErythrosineB-H2O-filtered-saturated-KI-quenched-ND3_2.hdf5
Photon-HDF5 WARNING: Missing field "author" in "/identity".
Photon-HDF5 WARNING: Missing field "author_affiliation" in "/identity".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data0".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data1".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data2".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data3".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data4".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data5".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data6".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data7".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data8".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data9".
P

In [16]:
basepath = r'E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-10-13'

In [17]:
datafile = Path(basepath, r'IRF-glass-air-BP-ND3_5.dat')

In [18]:
assert datafile.exists()

In [19]:
save_polimi_tcspc_to_photon_hdf5(datafile, description='IRF measurement', 
                                 validate=True)

Saving: E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-10-13\IRF-glass-air-BP-ND3_5.hdf5
Photon-HDF5 WARNING: Missing field "author" in "/identity".
Photon-HDF5 WARNING: Missing field "author_affiliation" in "/identity".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data0".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data1".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data2".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data3".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data4".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data5".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data6".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data7".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data8".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data9".
Photon-HDF5 WARNING: Missing fie

In [20]:
datafile = Path(basepath, r'IRF-glass-air-LP_BP_4.dat')

In [21]:
assert datafile.exists()

In [22]:
save_polimi_tcspc_to_photon_hdf5(datafile, description='IRF measurement', 
                                 validate=True)

Saving: E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-10-13\IRF-glass-air-LP_BP_4.hdf5
Photon-HDF5 WARNING: Missing field "author" in "/identity".
Photon-HDF5 WARNING: Missing field "author_affiliation" in "/identity".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data0".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data1".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data2".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data3".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data4".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data5".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data6".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data7".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data8".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data9".
Photon-HDF5 WARNING: Missing fiel

In [23]:
datafile = Path(basepath, r'IRF-glass-air-ND1_BP_2.dat')

In [24]:
assert datafile.exists()

In [25]:
save_polimi_tcspc_to_photon_hdf5(datafile, description='IRF measurement', 
                                 validate=True)

Saving: E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-10-13\IRF-glass-air-ND1_BP_2.hdf5
Photon-HDF5 WARNING: Missing field "author" in "/identity".
Photon-HDF5 WARNING: Missing field "author_affiliation" in "/identity".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data0".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data1".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data2".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data3".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data4".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data5".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data6".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data7".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data8".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data9".
Photon-HDF5 WARNING: Missing fie

In [26]:
datafile = Path(basepath, r'ErythrosinB_high_conc_1.dat')

In [27]:
assert datafile.exists()

In [28]:
sample = {'sample_name': 'ErythrosineB at high concentration .',
          'num_dyes': 1,
          'dye_names': 'ErythrosineB',
          'buffer_name': 'Filtered water'}

setup = {'excitation_input_powers': [0.1]}

description = 'High-concentraction of ErythrosineB for testing short lifetimes.'

In [29]:
save_polimi_tcspc_to_photon_hdf5(datafile, sample=sample, setup=setup, description=description, 
                                 validate=True)

Saving: E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-10-13\ErythrosinB_high_conc_1.hdf5
Photon-HDF5 WARNING: Missing field "author" in "/identity".
Photon-HDF5 WARNING: Missing field "author_affiliation" in "/identity".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data0".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data1".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data2".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data3".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data4".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data5".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data6".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data7".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data8".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data9".
Photon-HDF5 WARNING: Missing fi

In [30]:
datafile = Path(basepath, r'ErythrosinB_sample2_3.dat')

In [31]:
assert datafile.exists()

In [32]:
sample = {'sample_name': 'ErythrosineB at high concentration .',
          'num_dyes': 1,
          'dye_names': 'ErythrosineB',
          'buffer_name': 'Filtered water'}

setup = {'excitation_input_powers': [0.1]}

description = 'High-concentraction of ErythrosineB for testing short lifetimes.'

In [33]:
save_polimi_tcspc_to_photon_hdf5(datafile, sample=sample, setup=setup, description=description, 
                                 validate=True)

Saving: E:\Data\Antonio\data\POLIMI-TCSPC-16ch\2015-10-13\ErythrosinB_sample2_3.hdf5
Photon-HDF5 WARNING: Missing field "author" in "/identity".
Photon-HDF5 WARNING: Missing field "author_affiliation" in "/identity".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data0".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data1".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data2".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data3".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data4".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data5".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data6".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data7".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data8".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data9".
Photon-HDF5 WARNING: Missing fiel

# 4. Batch conversion for selected files

In [5]:
path = Path(basepath, r'Antonio\data\2015-09-08')

In [6]:
metapath = path.joinpath('metadata.yaml')
metapath.exists()

True

In [7]:
metadata = yaml.load(metapath.open())

In [8]:
print(yaml.dump(metadata, default_flow_style=False))

DA_12_D_8_MIX_LP-BP-100mW:
  description: 'Mixture of 60% FRET, 40% DO sample, 60uL, 66pM (total). No gasket.

    '
  sample_da:
    buffer_name: TE50
    dye_names: ATTO550, ATTO647N
    num_dyes: 2
    sample_name: Mixture 60% FRET TA(-15)-NTD(-3), 40% DO T-NTD(-3)80bp, 66pM total.
  setup: &id001
    excitation_input_powers:
    - 0.1
IRF-TE50-surface-BP-100mW_3:
  description: ''
  setup: *id001
IRF-TE50-surface-plus-BP-100mW_4:
  description: ''
  setup: *id001
T-NTD-3-66pM-60uL-100mW_2:
  description: ''
  sample: &id002
    buffer_name: TE50
    dye_names: ATTO550
    num_dyes: 1
    sample_name: 80bp dsDNA labeled with ATTO550, T-NTD(-3)
  setup: *id001
TA-15-NT-3_LP-BP-100mW_5:
  description: 'Medium FRET sample 20uL no gasket.

    '
  sample: &id003
    buffer_name: TE50
    dye_names: ATTO550, ATTO647N
    num_dyes: 2
    sample_name: 80bp dsDNA, TA(-15)-NTD(-3)
  setup: *id001
sample_d: *id002
sample_da: *id003
setup: *id001



In [9]:
datafiles = [f for f in path.glob('*.dat')]# if 'DNA' not in str(f)]
#datafiles

In [10]:
metadata_paths = [Path(path, name).with_suffix('.dat') for name in metadata.keys()]
for file in datafiles:
    print('- ', file, flush=True)
    
    meta = dict(sample=None, setup=None, description=None)
    if file in metadata_paths:
        meta = metadata[file.stem]
    print('  * META: %s' % meta, flush=True)
    save_polimi_tcspc_to_photon_hdf5(file, validate=True, **meta)

-  C:\Google Drive on Deliquescent\Data on Deliquescent\Antonio\data\2015-09-08\DA_12_D_8_MIX_LP-BP-100mW_6.dat
  * META: {'description': None, 'setup': None, 'sample': None}
Saving: C:\Google Drive on Deliquescent\Data on Deliquescent\Antonio\data\2015-09-08\DA_12_D_8_MIX_LP-BP-100mW_6.hdf5
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data0".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data1".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data10".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data11".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data12".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data13".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data14".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data15".
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data2".
Photon-HDF5 WARNING: Missi

# 5. Batch conversion of all files

In [ ]:
for file in Path(basepath).glob('**/*.dat'):
    if not polimi_tcspc.is_polimi_tcspc(str(file)): 
        continue

    print('\n - %s' % Path(*file.parts[3:]), flush=True)
    
    if file.with_suffix('.hdf5').exists(): 
       print('- HDF5 found, skipping.', flush=True)
       continue
        
    meta = dict(sample=None, setup=None, description=None)
    metapath = file.parent.joinpath('metadata.yaml')
    if metapath.exists():
        metadata = yaml.load(metapath.open())
        metadata_paths = [Path(metapath.parent, name).with_suffix('.dat') for name in metadata.keys()]
        if file in metadata_paths:
            meta = metadata[file.stem]
    print('  * META: %s' % meta, flush=True)
    #save_polimi_tcspc_to_photon_hdf5(filename)

In [6]:
print('OK')

OK
